<a href="https://colab.research.google.com/github/veterydaisy/Proyecto-2-ChurnInsight-Predicci-n-de-Cancelaci-n-de-Clientes-grupo-H12-25-L-Equipo-11-Data-Science/blob/main/preparacion_de_las_bases_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Lo que haremos para preparar las bases de datos del Proyecto**

**La Fuente Real:** "Orange Telecom Churn Dataset" (Hosted en mlcourse.ai)
Esta base de datos proviene de una operadora de telecomunicaciones real (Orange). Es ampliamente usada en la industria para benchmarks.

**¿Es real?** Sí, son registros de actividad de clientes reales.

**¿Es estable?** El enlace que usaré a continuación es del repositorio del curso mlcourse.ai, que es extremadamente fiable y rara vez se cae.

**Vamos a hacer lo siguiente:**

1.- Descargar los datos reales desde este servidor estable.

2.- Mapear las columnas reales a los nombres que pidió el cliente.

3.- "Romper" el archivo en 3 partes (Clientes, Uso, Finanzas) para poder hacer el trabajo de ETL.

4.- Script Final (Datos Reales + Preparación de Entorno)
Usará la librería pandas para descargar directo sin tener que buscar botones de descarga.

In [ ]:
import pandas as pd
import numpy as np

print("--- DESCARGANDO DATOS REALES (ORANGE TELECOM) ---")

# 1. ENLACE ESTABLE (Repositorio de mlcourse.ai)
# Estos son datos reales de comportamiento de usuarios
url_real = "https://raw.githubusercontent.com/yorko/mlcourse.ai/master/data/telecom_churn.csv"

try:
    df_real = pd.read_csv(url_real)
    print(f"✅ Datos descargados exitosamente. Dimensiones originales: {df_real.shape}")
except Exception as e:
    print("❌ Error de conexión. Verifica tu internet.")
    # Si fallara

# 2. MAPEO A REQUERIMIENTOS DEL CLIENTE
# Tomamos las columnas reales y las renombramos a lo que pidieron
df_proyecto = pd.DataFrame()

df_proyecto['customer_id'] = [f'REAL-{i+1000}' for i in range(len(df_real))] # Generamos ID
df_proyecto['tenure'] = df_real['Account length'] # Antigüedad real en días
df_proyecto['contract_type'] = np.where(df_real['International plan'] == 'Yes', 'Two Year', 'Month-to-Month')
df_proyecto['subscription_type'] = np.where(df_real['Voice mail plan'] == 'Yes', 'Premium', 'Basic')

# Aquí está: COMPORTAMIENTO REAL
# Sumamos minutos de día, tarde y noche para tener el uso total real
df_proyecto['usage_time'] = df_real['Total day minutes'] + df_real['Total eve minutes'] + df_real['Total night minutes']
df_proyecto['login_frequency'] = df_real['Total day calls'] # Proxy: Frecuencia de uso (llamadas)

# Datos Financieros Reales
df_proyecto['total_spend'] = df_real['Total day charge'] + df_real['Total eve charge'] + df_real['Total night charge']
# Simulamos historial de pago basado en reclamos (Dato real: Customer service calls)
df_proyecto['payment_record'] = np.where(df_real['Customer service calls'] > 2, 'Delayed', 'Good')

df_proyecto['churn'] = df_real['Churn'].astype(int)

# 3. FILTRADO (Para cumplir el requisito de < 2000 filas)
# Tomamos una muestra aleatoria de 1800 clientes reales
df_final = df_proyecto.sample(n=1800, random_state=42)

# 4. CREACIÓN DEL ESCENARIO ETL (ROMPIENDO LOS DATOS)
# Separamos en 3 archivos para practicar el JOIN y la LIMPIEZA

# A. Tabla Clientes (CRM)
df_clientes = df_final[['customer_id', 'tenure', 'contract_type', 'subscription_type']].copy()
# Introducimos suciedad: 50 nulos en tenure
df_clientes.loc[df_clientes.sample(n=50).index, 'tenure'] = np.nan

# B. Tabla Uso (Logs)
df_logs = df_final[['customer_id', 'usage_time', 'login_frequency']].copy()

# C. Tabla Facturación
df_facturacion = df_final[['customer_id', 'payment_record', 'total_spend', 'churn']].copy()
# Introducimos suciedad: convertir 'total_spend' a string con símbolo '$' en algunos casos
# Esto te obligará a limpiar la columna antes de usarla
indices_sucios = df_facturacion.sample(n=20).index
df_facturacion.loc[indices_sucios, 'total_spend'] = df_facturacion.loc[indices_sucios, 'total_spend'].apply(lambda x: f"${x:.2f}")

# GUARDAR EN PC
df_clientes.to_csv('base_clientes_real.csv', index=False)
df_logs.to_csv('base_logs_real.csv', index=False)
df_facturacion.to_csv('base_facturacion_real.csv', index=False)

print("\n✅ ¡LISTO! Se han generado 3 archivos CSV con DATOS REALES:")
print("1. base_clientes_real.csv (Contiene nulos en tenure)")
print("2. base_logs_real.csv (Datos de uso reales)")
print("3. base_facturacion_real.csv (Contiene signos '$' que se deben limpiar)")

--- DESCARGANDO DATOS REALES (ORANGE TELECOM) ---
✅ Datos descargados exitosamente. Dimensiones originales: (3333, 20)

✅ ¡LISTO! Se han generado 3 archivos CSV con DATOS REALES:
1. base_clientes_real.csv (Contiene nulos en tenure)
2. base_logs_real.csv (Datos de uso reales)
3. base_facturacion_real.csv (Contiene signos '$' que se deben limpiar)


/tmp/ipython-input-927337654.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['$57.44' '$64.72' '$77.78' '$55.07' '$50.49' '$72.67' '$57.38' '$56.81'
 '$48.84' '$59.75' '$42.52' '$52.10' '$69.68' '$40.81' '$57.48' '$34.74'
 '$57.76' '$41.95' '$55.14' '$62.12']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_facturacion.loc[indices_sucios, 'total_spend'] = df_facturacion.loc[indices_sucios, 'total_spend'].apply(lambda x: f"${x:.2f}")
